In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("raw_data.csv")

slim = df[[
    "Reference area", "Commodity", "TIME_PERIOD", "Measure", "OBS_VALUE", "Unit of measure"
]].rename(columns={
    "Reference area": "country",
    "Commodity": "commodity",
    "TIME_PERIOD": "year",
    "Measure": "measure",
    "OBS_VALUE": "value",
    "Unit of measure": "unit"
})

print(slim.shape)
print(slim["measure"].unique())
print(slim["unit"].unique())
print(slim["year"].min(), slim["year"].max())


(20900, 6)
['Ending stocks' 'Consumption' 'Exports' 'Imports' 'Production' 'Yield']
['Tonnes' 'Tonnes per hectare']
2025 2034


In [2]:
wide = slim.pivot(
    index=["country", "commodity", "year"],
    columns="measure",
    values="value"
).reset_index()

wide.columns.name = None

print(wide.shape)
print(wide.head())


(3800, 9)
     country      commodity  year  Consumption  Ending stocks    Exports  \
0  Argentina  Beef and veal  2025    2315.6964            0.0   966.3682   
1  Argentina  Beef and veal  2026    2304.4448            0.0   981.8708   
2  Argentina  Beef and veal  2027    2293.8261            0.0  1003.6780   
3  Argentina  Beef and veal  2028    2287.4584            0.0  1025.7691   
4  Argentina  Beef and veal  2029    2277.2539            0.0  1040.4424   

   Imports  Production  Yield  
0   7.0182   3275.0464    NaN  
1   7.0372   3279.2783    NaN  
2   7.0114   3290.4928    NaN  
3   6.9866   3306.2409    NaN  
4   7.0125   3310.6838    NaN  


In [3]:
# כפילויות
dups = wide.duplicated(subset=["country", "commodity", "year"]).sum()
print("Duplicate rows:", dups)

# ערכים שליליים (לא אמורים להיות)
quant_cols = ["Production", "Consumption", "Imports", "Exports", "Ending stocks"]
for c in quant_cols:
    if c in wide.columns:
        print(c, (wide[c] < 0).sum())


Duplicate rows: 0
Production 0
Consumption 0
Imports 0
Exports 0
Ending stocks 0


In [4]:
wide = wide.rename(columns={
    "Production": "production",
    "Consumption": "consumption",
    "Imports": "imports",
    "Exports": "exports",
    "Ending stocks": "ending_stocks",
    "Yield": "yield"
})


In [5]:
wide = wide.sort_values(["country", "commodity", "year"]).reset_index(drop=True)
wide.head()


,country,commodity,year,consumption,ending_stocks,exports,imports,production,yield
0,Argentina,Beef and veal,2025,2315.6964,0.0,966.3682,7.0182,3275.0464,NaN
1,Argentina,Beef and veal,2026,2304.4448,0.0,981.8708,7.0372,3279.2783,NaN
2,Argentina,Beef and veal,2027,2293.8261,0.0,1003.6780,7.0114,3290.4928,NaN
3,Argentina,Beef and veal,2028,2287.4584,0.0,1025.7691,6.9866,3306.2409,NaN
4,Argentina,Beef and veal,2029,2277.2539,0.0,1040.4424,7.0125,3310.6838,NaN


In [6]:
wide["trade_balance"] = wide["exports"] - wide["imports"]

wide["self_sufficiency"] = np.where(
    wide["consumption"] > 0,
    wide["production"] / wide["consumption"],
    np.nan
)
wide["self_sufficiency_capped"] = wide["self_sufficiency"].clip(upper=5)

print(wide[["trade_balance", "self_sufficiency", "self_sufficiency_capped"]].describe())


       trade_balance  self_sufficiency  self_sufficiency_capped
count    3800.000000       3040.000000              3040.000000
mean      434.638226        267.479404                 0.910981
std     16067.316819       4662.747806                 0.862564
min   -164018.444500          0.000000                 0.000000
25%      -750.604550          0.389302                 0.389302
50%       -19.041600          0.913457                 0.913457
75%        13.436100          1.046104                 1.046104
max    164018.444600      93709.000000                 5.000000


In [7]:
wide.to_csv("processed_data.csv", index=False)

In [8]:
prod_by_commodity = (
    wide
    .groupby("commodity")["production"]
    .sum()
    .sort_values(ascending=False)
)

prod_by_commodity


,production
commodity,
Maize (corn),2.554819e+07
Raw milk from bovine animals,1.944626e+07
Wheat,1.628585e+07
Rice,1.037503e+07
Soya beans,8.585705e+06
Poultry meat,3.114313e+06
Pigmeat,2.490976e+06
Beef and veal,1.474772e+06
"Cheese, fresh or processed",5.367663e+05


In [9]:
wide["trade_balance"].describe()


,trade_balance
count,3800.000000
mean,434.638226
std,16067.316819
min,-164018.444500
25%,-750.604550
50%,-19.041600
75%,13.436100
max,164018.444600


In [10]:
(
    wide
    .groupby("country")["trade_balance"]
    .sum()
    .sort_values()
    .head(5)
)

,trade_balance
country,
Developing countries,-2.575196e+06
China (People’s Republic of),-1.444456e+06
Mexico,-3.383840e+05
Egypt,-2.869107e+05
Japan,-2.865478e+05


In [11]:
(
    wide
    .groupby("country")["trade_balance"]
    .sum()
    .sort_values(ascending=False)
    .head(5)
)

,trade_balance
country,
Developed countries,2.576989e+06
Brazil,1.771711e+06
United States,1.343586e+06
Russia,5.745317e+05
Argentina,5.682632e+05


In [12]:
wide["self_sufficiency_capped"].describe()


,self_sufficiency_capped
count,3040.000000
mean,0.910981
std,0.862564
min,0.000000
25%,0.389302
50%,0.913457
75%,1.046104
max,5.000000


In [13]:
wide.groupby("commodity")["yield"].count().sort_values(ascending=False)


,yield
commodity,
Soya beans,380
Maize (corn),380
Wheat,380
Raw milk from bovine animals,380
Rice,380
Beef and veal,0
"Cheese, fresh or processed",0
Raw cane or beet sugar,0
Pigmeat,0


In [14]:
top_producers = (
    wide
    .groupby("country")["production"]
    .sum()
    .sort_values(ascending=False)
    .head(10)
)

top_producers


,production
country,
Developing countries,2.868905e+07
Developed countries,1.730367e+07
China (People’s Republic of),7.528932e+06
United States,7.405270e+06
India,6.246681e+06
European Union,4.012248e+06
Brazil,3.910578e+06
Russia,1.679379e+06
Argentina,1.532495e+06


In [15]:
prod_trends = (
    wide
    .groupby(["commodity", "year"])["production"]
    .sum()
    .reset_index()
)
prod_trends.groupby("commodity")["production"].describe()


,count,mean,std,min,25%,50%,75%,max
commodity,,,,,,,,
Beef and veal,10.0,1.474772e+05,4978.230601,1.405395e+05,1.430455e+05,1.478833e+05,1.515337e+05,1.541622e+05
"Cheese, fresh or processed",10.0,5.367663e+04,1565.580012,5.133880e+04,5.252263e+04,5.368304e+04,5.483393e+04,5.600163e+04
Maize (corn),10.0,2.554819e+06,85320.019728,2.431437e+06,2.491099e+06,2.553323e+06,2.618617e+06,2.682793e+06
Pigmeat,10.0,2.490976e+05,2681.656914,2.449525e+05,2.470445e+05,2.491423e+05,2.510691e+05,2.530291e+05
Poultry meat,10.0,3.114313e+05,13675.194975,2.904979e+05,3.016910e+05,3.116901e+05,3.216011e+05,3.314369e+05
Raw cane or beet sugar,10.0,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
Raw milk from bovine animals,10.0,1.944626e+06,103383.417718,1.793231e+06,1.864149e+06,1.944234e+06,2.021383e+06,2.099763e+06
Rice,10.0,1.037503e+06,27597.213091,1.004153e+06,1.011844e+06,1.037341e+06,1.058982e+06,1.078598e+06
Soya beans,10.0,8.585705e+05,26855.290338,8.199898e+05,8.381728e+05,8.594537e+05,8.786130e+05,8.979281e+05


In [16]:
ranking = (
    wide
    .groupby(["year", "country"])["production"]
    .sum()
    .reset_index()
    .sort_values(["year", "production"], ascending=[True, False])
)
ranking

,year,country,production
8,2025,Developing countries,2.678726e+06
7,2025,Developed countries,1.676840e+06
5,2025,China (People’s Republic of),7.349579e+05
36,2025,United States,7.216004e+05
12,2025,India,5.580650e+05
...,...,...,...
346,2034,Chile,6.677946e+03
373,2034,Switzerland,4.947800e+03
361,2034,Malaysia,4.354423e+03
357,2034,Israel,3.180916e+03


In [17]:
global_prod_trends = (
    wide
    .groupby(["commodity", "year"])["production"]
    .sum()
    .reset_index()
)

global_prod_trends.head()


,commodity,year,production
0,Beef and veal,2025,140539.4553
1,Beef and veal,2026,141455.0654
2,Beef and veal,2027,142397.1552
3,Beef and veal,2028,144990.5207
4,Beef and veal,2029,147079.6363


In [18]:
prod_share = (
    wide
    .groupby(["commodity", "country"])["production"]
    .sum()
    .reset_index()
)

prod_share.head()


,commodity,country,production
0,Beef and veal,Argentina,33192.4514
1,Beef and veal,Australia,27058.1458
2,Beef and veal,Brazil,92649.3618
3,Beef and veal,Canada,14614.4950
4,Beef and veal,Chile,2336.9563


In [19]:
top5_share = (
    prod_share
    .groupby("commodity")
    .apply(lambda x: x.sort_values("production", ascending=False)
                     .head(5)["production"].sum()
          / x["production"].sum())
)

top5_share


/tmp/ipython-input-3359028740.py:4: RuntimeWarning: invalid value encountered in scalar divide
  .apply(lambda x: x.sort_values("production", ascending=False)
/tmp/ipython-input-3359028740.py:4: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.sort_values("production", ascending=False)


,0
commodity,
Beef and veal,0.740944
"Cheese, fresh or processed",0.881896
Maize (corn),0.848014
Pigmeat,0.873563
Poultry meat,0.743787
Raw cane or beet sugar,NaN
Raw milk from bovine animals,0.804175
Rice,0.888885
Soya beans,0.902490


In [20]:
dev_trends = (
    wide[wide["country"].isin(["Developed countries", "Developing countries"])]
    .groupby(["country", "year"])["production"]
    .sum()
    .reset_index()
)

dev_trends.head()


,country,year,production
0,Developed countries,2025,1.676840e+06
1,Developed countries,2026,1.687265e+06
2,Developed countries,2027,1.698134e+06
3,Developed countries,2028,1.711303e+06
4,Developed countries,2029,1.724467e+06


In [21]:
trade_by_country = (
    wide
    .groupby("country")["trade_balance"]
    .sum()
    .sort_values()
)

trade_by_country.head(10), trade_by_country.tail(10)


(country
 Developing countries           -2.575196e+06
 China (People’s Republic of)   -1.444456e+06
 Mexico                         -3.383840e+05
 Egypt                          -2.869107e+05
 Japan                          -2.865478e+05
 Indonesia                      -2.469441e+05
 Korea                          -2.206278e+05
 Iran                           -2.047484e+05
 Viet Nam                       -1.597514e+05
 Philippines                    -1.506634e+05
 Name: trade_balance, dtype: float64,
 country
 Paraguay               1.231747e+05
 India                  2.289631e+05
 Australia              2.636889e+05
 Canada                 3.024427e+05
 Ukraine                4.381775e+05
 Argentina              5.682632e+05
 Russia                 5.745317e+05
 United States          1.343586e+06
 Brazil                 1.771711e+06
 Developed countries    2.576989e+06
 Name: trade_balance, dtype: float64)

In [22]:
trade_trends = (
    wide
    .groupby(["country", "year"])["trade_balance"]
    .sum()
    .reset_index()
)

trade_trends.head()


,country,year,trade_balance
0,Argentina,2025,53694.1062
1,Argentina,2026,54017.9373
2,Argentina,2027,56857.0918
3,Argentina,2028,57090.9814
4,Argentina,2029,57309.6542


In [23]:
dev_trade = (
    wide[wide["country"].isin(["Developed countries", "Developing countries"])]
    .groupby(["country", "year"])["trade_balance"]
    .sum()
    .reset_index()
)

dev_trade.head()


,country,year,trade_balance
0,Developed countries,2025,240914.9602
1,Developed countries,2026,244764.7341
2,Developed countries,2027,247750.7068
3,Developed countries,2028,251853.1253
4,Developed countries,2029,255702.0548


In [24]:
trade_by_commodity = (
    wide
    .groupby("commodity")["trade_balance"]
    .sum()
    .sort_values()
)

trade_by_commodity


,trade_balance
commodity,
Raw milk from bovine animals,0.0000
"Cheese, fresh or processed",4989.5384
Beef and veal,9739.2569
Pigmeat,17816.0509
Soya beans,63157.7775
Poultry meat,71480.3686
Raw cane or beet sugar,102329.9053
Rice,228346.4067
Maize (corn),297130.5176


In [25]:
ss_by_country = (
    wide
    .groupby("country")["self_sufficiency_capped"]
    .mean()
    .sort_values()
)

ss_by_country.head(10), ss_by_country.tail(10)


(country
 Malaysia          0.292867
 Saudi Arabia      0.337171
 Korea             0.385725
 Japan             0.386656
 Israel            0.412415
 Philippines       0.451734
 Viet Nam          0.495139
 United Kingdom    0.496300
 Norway            0.509547
 Chile             0.528475
 Name: self_sufficiency_capped, dtype: float64,
 country
 Kazakhstan       1.217999
 United States    1.288235
 Brazil           1.341608
 Argentina        1.567115
 India            1.567148
 New Zealand      1.570242
 Canada           1.709846
 Australia        1.710207
 Ukraine          1.835305
 Paraguay         2.174724
 Name: self_sufficiency_capped, dtype: float64)

In [26]:
yield_commodities = (
    wide.groupby("commodity")["yield"]
    .count()
    .sort_values(ascending=False)
)

yield_commodities


,yield
commodity,
Soya beans,380
Maize (corn),380
Wheat,380
Raw milk from bovine animals,380
Rice,380
Beef and veal,0
"Cheese, fresh or processed",0
Raw cane or beet sugar,0
Pigmeat,0


In [27]:
yield_by_country = (
    wide
    .dropna(subset=["yield"])
    .groupby("country")["yield"]
    .mean()
    .sort_values()
)

yield_by_country.head(10), yield_by_country.tail(10)


(country
 Nigeria        1.580690
 Norway         2.454840
 Thailand       2.787086
 Malaysia       2.846110
 India          2.876380
 Colombia       3.020234
 Ethiopia       3.074644
 Indonesia      3.092642
 Pakistan       3.299776
 Philippines    3.360168
 Name: yield, dtype: float64,
 country
 Egypt                  5.378338
 Switzerland            5.528148
 European Union         5.570320
 Türkiye                5.608474
 Chile                  5.663508
 Developed countries    5.735942
 Canada                 5.957844
 Saudi Arabia           6.055536
 United States          7.206570
 Israel                 9.662610
 Name: yield, dtype: float64)

In [28]:
yield_dev = (
    wide[
        (wide["country"].isin(["Developed countries", "Developing countries"])) &
        (~wide["yield"].isna())
    ]
    .groupby(["country", "year"])["yield"]
    .mean()
    .reset_index()
)

yield_dev.head()


,country,year,yield
0,Developed countries,2025,5.60342
1,Developed countries,2026,5.63130
2,Developed countries,2027,5.66054
3,Developed countries,2028,5.69130
4,Developed countries,2029,5.72080


In [29]:
yield_trends = (
    wide
    .dropna(subset=["yield"])
    .groupby(["commodity", "year"])["yield"]
    .mean()
    .reset_index()
)

yield_trends.head()


,commodity,year,yield
0,Maize (corn),2025,6.682889
1,Maize (corn),2026,6.747682
2,Maize (corn),2027,6.825576
3,Maize (corn),2028,6.903511
4,Maize (corn),2029,6.979947


In [31]:
# ============================================
# Colab cell 3: The exact checks you need for the report
# ============================================
n_rows = len(wide)
n_countries = wide["country"].nunique(dropna=True)
n_commodities = wide["commodity"].nunique(dropna=True)
min_year = int(wide["year"].min())
max_year = int(wide["year"].max())
n_years = wide["year"].nunique(dropna=True)

print("===== REPORT NUMBERS =====")
print("Rows (records):", n_rows)
print("Countries:", n_countries)
print("Commodities:", n_commodities)
print("Years count:", n_years)
print("Year range:", f"{min_year}-{max_year}")


===== REPORT NUMBERS =====
Rows (records): 3800
Countries: 38
Commodities: 10
Years count: 10
Year range: 2025-2034
